In [ ]:
import fs from 'fs';
import bs58 from 'bs58';
import { Keypair } from '@solana/web3.js';

function getOrCreateKeypair(dir: string, keyName: string): Keypair {
  if (!fs.existsSync(dir)) fs.mkdirSync(dir, { recursive: true });

  const authorityKey = `${dir}/${keyName}.json`;

  if (fs.existsSync(authorityKey)) {
    const data: { secretKey: string; publicKey: string } = JSON.parse(fs.readFileSync(authorityKey, 'utf-8'));
    return Keypair.fromSecretKey(bs58.decode(data.secretKey));
  } else {
    const keypair = Keypair.generate();
    fs.writeFileSync(
      authorityKey,
      JSON.stringify({
        secretKey: bs58.encode(keypair.secretKey),
        publicKey: keypair.publicKey.toBase58(),
      })
    );
    return keypair;
  }
}

async function MakeWallets(amount: number, dir: string) {
  for (let i = 0; i < amount; i++) {
    await getOrCreateKeypair(dir, `account${i + 1}`);
  }
}

// Directory to store the wallets
const KEYS_FOLDER = './solana_wallets';

// Generate 10 Solana wallets (adjust as needed)
MakeWallets(10, KEYS_FOLDER);


In [ ]:
import os
import time
import logging
import random
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Path to Solana wallets
KEYS_FOLDER = './solana_wallets'

# Target URL
target_url = "https://pump.fun/A6M7D6Y4cb7HNiVMRd6M2Ke5fWCeBU2fBXrviVihpump"

# List of user agents to randomize the browser identities
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
]

# Load Solana wallets
def load_wallets(dir: str):
    wallets = []
    for filename in os.listdir(dir):
        if filename.endswith(".json"):
            with open(os.path.join(dir, filename), 'r') as f:
                wallet = json.load(f)
                wallets.append(wallet)
    return wallets

wallets = load_wallets(KEYS_FOLDER)
wallet_index = 0

# Initialize the WebDriver for Selenium
def initialize_driver(user_agent):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"user-agent={user_agent}")
    options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# Post positive messages to live chat
def post_positive_message(driver, wallet):
    positive_messages = [
        "Great stream!",
        "This is awesome!",
        "Loving the content!",
        "Keep up the good work!",
        "This livestream is amazing!"
    ]
    message = random.choice(positive_messages)
    
    try:
        chat_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".chat-input-selector"))  # Update with actual selector
        )
        chat_input.send_keys(f"{message} - {wallet['publicKey']}")
        chat_input.send_keys("\n")
        logging.info(f"Sent message: {message} - {wallet['publicKey']}")
    except Exception as e:
        logging.error(f"Failed to post message with wallet {wallet['publicKey']}: {e}")

# Open a browser, click "join livestream," and post messages
def browse_page(browser_id, retries=3):
    global wallet_index
    user_agent = random.choice(USER_AGENTS)
    driver = initialize_driver(user_agent)
    
    try:
        logging.info(f"Browser {browser_id} with user-agent {user_agent} opened.")
        driver.get(target_url)
        logging.info(f"Browser {browser_id}: Navigated to {target_url}.")

        # Random sleep to simulate natural browsing behavior
        time.sleep(random.uniform(2, 5))

        # Wait for the "join livestream" button and click it
        join_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.inline-flex.items-center.justify-center.whitespace-nowrap.rounded-md.font-medium.ring-offset-white.transition-colors.focus-visible\\:outline-none.focus-visible\\:ring-2.focus-visible\\:ring-slate-950.focus-visible\\:ring-offset-2.disabled\\:pointer-events-none.disabled\\:opacity-50.dark\\:ring-offset-slate-950.dark\\:focus-visible\\:ring-slate-300.dark\\:hover\\:bg-slate-800.dark\\:hover\\:text-slate-50.h-10.px-4.py-2.-mt-5.text-2xl.text-slate-50.hover\\:font-bold.hover\\:bg-transparent.hover\\:text-slate-50.cursor-pointer"))
        )
        join_button.click()
        logging.info(f"Browser {browser_id}: Clicked 'join livestream' button.")

        # Wait for the livestream page to load by checking for a video element
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "video.lk-participant-media-video"))
        )
        logging.info(f"Browser {browser_id}: Livestream loaded successfully.")

        # Post a positive message in the chat using a Solana wallet
        if wallet_index < len(wallets):
            post_positive_message(driver, wallets[wallet_index])
            wallet_index += 1
        else:
            logging.warning(f"Browser {browser_id}: No more wallets available to post messages.")

        logging.info(f"Browser {browser_id}: Staying on livestream for 1 hour.")
        time.sleep(3600)  # Stay on the page for 1 hour

        logging.info(f"Browser {browser_id}: Completed session.")

    except Exception as e:
        logging.error(f"Browser {browser_id}: Error during browsing: {e}")
        if retries > 0:
            logging.info(f"Browser {browser_id}: Retrying... ({retries} attempts left)")
            driver.quit()
            browse_page(browser_id, retries - 1)  # Retry the process
    finally:
        driver.quit()
        logging.info(f"Browser {browser_id}: Closed.")

# Open multiple browser instances
def open_multiple_browsers(num_browsers=50):
    with ThreadPoolExecutor(max_workers=num_browsers) as executor:
        for i in range(num_browsers):
            executor.submit(browse_page, i + 1)

# Start the browsing session
open_multiple_browsers()

In [ ]:
import os
import time
import logging
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Target URL
target_url = "https://pump.fun/pWP9rfrZzf3yFz6D7fRjYrVLcqmRN98gVRGTS6Vpump"

# List of user agents to randomize the browser identities
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
]

# Initialize the WebDriver for Selenium
def initialize_driver(user_agent):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"user-agent={user_agent}")
    options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# Open a browser, click "join livestream," and stay on the target page for a specified duration
def browse_page(browser_id, retries=3):
    user_agent = random.choice(USER_AGENTS)
    driver = initialize_driver(user_agent)
    
    try:
        logging.info(f"Browser {browser_id} with user-agent {user_agent} opened.")
        driver.get(target_url)
        logging.info(f"Browser {browser_id}: Navigated to {target_url}.")

        # Random sleep to simulate natural browsing behavior
        time.sleep(random.uniform(2, 5))

        # Wait for the "join livestream" button and click it
        join_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.inline-flex.items-center.justify-center.whitespace-nowrap.rounded-md.font-medium.ring-offset-white.transition-colors.focus-visible\\:outline-none.focus-visible\\:ring-2.focus-visible\\:ring-slate-950.focus-visible\\:ring-offset-2.disabled\\:pointer-events-none.disabled\\:opacity-50.dark\\:ring-offset-slate-950.dark\\:focus-visible\\:ring-slate-300.dark\\:hover\\:bg-slate-800.dark\\:hover\\:text-slate-50.h-10.px-4.py-2.-mt-5.text-2xl.text-slate-50.hover\\:font-bold.hover\\:bg-transparent.hover\\:text-slate-50.cursor-pointer"))
        )
        join_button.click()
        logging.info(f"Browser {browser_id}: Clicked 'join livestream' button.")

        logging.info(f"Browser {browser_id}: Staying on page for 1 hour.")
        time.sleep(3600)  # Stay on the page for 1 hour

        logging.info(f"Browser {browser_id}: Completed session.")

    except Exception as e:
        logging.error(f"Browser {browser_id}: Error during browsing: {e}")
        if retries > 0:
            logging.info(f"Browser {browser_id}: Retrying... ({retries} attempts left)")
            driver.quit()
            browse_page(browser_id, retries - 1)  # Retry the process
    finally:
        driver.quit()
        logging.info(f"Browser {browser_id}: Closed.")

# Open multiple browser instances
def open_multiple_browsers(num_browsers=50):
    with ThreadPoolExecutor(max_workers=num_browsers) as executor:
        for i in range(num_browsers):
            executor.submit(browse_page, i + 1)

# Start the browsing session
open_multiple_browsers()